<a href="https://colab.research.google.com/github/mei-yong/realtime_architecture/blob/main/Delta_Live_Table_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### This is a test title